# Problem Set 2 - Part 1: Ambiguity and Inference

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the following instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).


### Pre-requisite knowledge

Understanding of `problem-set-1`:
- First order logic
- Lambda calculus
- Feature unification context free grammar

### Instructions

- Follow the instructions step-by-step.
- Run all cells before modifying them to see how the code works.
- The tasks will ask you to modify cells in this notebook.
- Modify the cell to provide your answer and run a test before submiting the file.

In [ ]:
# This task requires NLTK and Jupyter Notebook (IPython package).

import nltk
from nltk.grammar import FeatureGrammar
from nltk.sem import cooper_storage as cs

from utils import display_latex, display_translation, display_tree, display, Markdown
from copy import deepcopy

read_expr = nltk.sem.Expression.fromstring

Run the following cell to import functions and syntax specific to this problem set:

In [ ]:
from utils2 import sem_parser, evaluate_sentences, syntax, syntax_notv

### 1. Ambiguity in quantifiers [6 marks in total]

Follow the descriptions and instructions in sections **(1.1)**, **(1.2)** and **(1.3)** to learn about the code and the examples. Then answer the questions in section **(1.4)**.

#### 1.1. FCFG with a SEM feature

You should already be familar with FCFG with a SEM feature. 
Use the code below to parse the following sentences to their semantic representations:

1. every dog bites a bone
2. a man gives a bone to every dog
3. Russia gives Moscow to Napoleon

In [ ]:
sentences = [
    'all dogs bite a bone',
    'a man gives a bone to every dog',
    'Russia gives Moscow to Napoleon',
]

# if you want to see the parse tree change `verbose` to True:
sents_reps = sem_parser(sentences, syntax, verbose=False)

for i, (sent, semreps) in enumerate(sents_reps.items()):
    for semrep in semreps:
        display_translation(f"{i+1}. {sent}", semrep)

#### 1.2. Evaluate sentences in a model

We have a world model with entities and sets to represent their properties and relationships. 
Run the code below to evaluate the sentence representations from the previous section in the world model defined below.

In [ ]:
# a world model:
entities = """
russia    => c1
moscow    => c2
napoleon  => m2
"""
unaries = """
dog  => {d1, d2, d3} 
man  => {m1, m2} 
bone => {bn1, bn2} 
"""
binaries = """
bite => {(d1, bn1), (d2, bn2), (d3, bn2)} 
"""
ternaries = """
give => {(c1, c2, m2), (m1, bn1, d1), (m1, bn2, d2), (m1, bn2, d3)} 
"""

# world is a collection of entities and relations:
world = entities + unaries + binaries + ternaries

# parse the sentences:
sents_reps = sem_parser(sentences, syntax, verbose=False)
# evaluate them:
sents_reps_vals = evaluate_sentences(sents_reps, world)

# print all readings
for i, (sent, semreps_vals) in enumerate(sents_reps_vals.items()):
    for semrep, val in semreps_vals.items():
        print(f"{i+1}. {sent}:")
        display_translation(val, semrep)

#### 1.3. Exploiting syntactic ambiguity to represent semantic ambiguity

Ditransitive verbs (`DTV`) have two objects. Use the code below to inspect the lexical representation of `give(s)` in the grammar. When constructing a `VP`, we take the semantic representation of a di-trasitive verb `DTV`as a function and apply it on both direct and indirect objects: `vp = ?dtv(?obj,?pp)`. The lambda bound variables `Y` and `X` are substituted with `?obj` and `?pp`.

In [ ]:
print(syntax.productions()[-1])
print(syntax.productions()[-5])

In the lecture we discussed that without a mechanism such as Cooper storage we would only get one scoping of quanitifers.

One trick to ensure a different scoping for `give(s)` is  to create a parallel syntactic rule that contains a different internal order of composition of arguments in the `SEM`feature as follows: 

For example, we can change the compositional function from:
$$\lambda Y\ X\ x.X(\lambda z.Y(\lambda y.give(x,y,z)))$$
to:
$$\lambda Y\ X\ x.Y(\lambda y.X(\lambda z.give(x,y,z)))$$

As we now have two rules for `DTV` this will introduce syntactic ambiguity when a parses encouters words such `give(s)`.
The following code adds two alternative rules to those in the previous cell. Run the code to see the parse results for sentences and their evaluation in the model.

In [ ]:
fcfg_string_give = r"""
DTV[NUM=sg,SEM=<\Y X x.Y(\y.X(\z.give(x,y,z)))>,TNS=pres] -> 'gives'
DTV[NUM=pl,SEM=<\Y X x.Y(\y.X(\z.give(x,y,z)))>,TNS=pres] -> 'give'
"""

# this is going to add new rules to the syntax:
new_syntax = FeatureGrammar(
    syntax.start(),
    syntax.productions() + FeatureGrammar.fromstring(fcfg_string_give).productions()
)

# parse the sentences with new syntax:
sents_reps = sem_parser(sentences, new_syntax, verbose=False)
# evaluate them:
sents_reps_vals = evaluate_sentences(sents_reps, world)

for i, (sent, semreps_vals) in enumerate(sents_reps_vals.items()):
    for semrep, val in semreps_vals.items():
        print(f"{i+1}. {sent}:")
        display_translation(val, semrep)

#### 1.4. Questions

**1a.** Because the word `give`can now be expanded following two rules we get two extra parse trees for the second and and the third sentence. Answer the following questions: **[2 marks]**
   - Why sentence (2) with `give` has two different readings but sentence (3) has only one reading?
   - What is the difference between the two representations of sentence (2)?

**1b.** When you evalute sentence (2) in the model above, one reading is True and the other is False. Change the part of the model marked with `???` so that all readings all sentences will be True. **[1 marks]** 

Explain why the readings are true. **[+1 mark]**

In [ ]:
ternaries = """
give => {(c1, c2, m2), ???}
"""

# new world
world = entities + unaries + binaries + ternaries

# evaluate them:
sents_reps_vals = evaluate_sentences(sents_reps, world)

for sent, semreps_vals in sents_reps_vals.items():
    for semrep, val in semreps_vals.items():
        print(f"{sent}:")
        display_translation(val, semrep)

**1c.** Consider a world with a `bite` relation as shown below.
Write *another* representation of the sentence "all dogs bite a bone" 
that is different from the one you get above and that is *True* in this model. 
Write your answer in `???`. **[1 mark]** 

In [ ]:
binaries = """
bite => {(d1, bn1), (d2, bn1), (d3, bn1)} 
"""

# new world
world = entities + unaries + binaries + ternaries

sents_reps_copy = deepcopy(sents_reps)
sents_reps_copy['all dogs bite a bone'].append(
    read_expr(r"???")
)

# evaluate them:
sents_reps_vals_copy = evaluate_sentences(sents_reps_copy, world)

for i, (sent, semreps_vals) in enumerate(sents_reps_vals_copy.items()):
    if sent == 'all dogs bite a bone':
        for semrep, val in semreps_vals.items():
            print(f"{i+1}. {sent}:")
            display_translation(val, semrep)

**1d.** Consider a new set of tuples representing the `give` predicate below.
Write *another* representation for "a man gives a bone to every dog"
that is *True* in the updated model.
Write your answer in `???`. **[1 mark]** 

In [ ]:
ternaries = """
give => {(c1, c2, m2), (m2, bn1, d1), (m1, bn1, d2), (m1, bn1, d3)}
"""

# new world
world = entities + unaries + binaries + ternaries

sents_reps_copy = deepcopy(sents_reps)
sents_reps_copy['a man gives a bone to every dog'].append(
    read_expr(r"???")
)

# evaluate them:
sents_reps_vals_copy = evaluate_sentences(sents_reps_copy, world)

for i, (sent, semreps_vals) in enumerate(sents_reps_vals_copy.items()):
    if sent == 'a man gives a bone to every dog':
        for semrep, val in semreps_vals.items():
            print(f"{i+1}. {sent}:")
            display_translation(val, semrep)

### 2. Cooper storage [8 marks in total]

#### 2.1. Learn about CooperSotrage
Study the following grammar with a `SEM` that is split between `CORE` and `STORE`. However, in this example, we never add anything to the `STORE`.

In [ ]:
fcfg_storage_base = r"""
% start S

S[SEM=[CORE=<?subj(?vp)>, STORE=(?b1+?b2)]] -> NP[NUM=?n,SEM=[CORE=?subj, STORE=?b1]] VP[NUM=?n,SEM=[CORE=?vp, STORE=?b2]]

Nom[NUM=?n,SEM=?s] -> N[NUM=?n,SEM=?s]

VP[NUM=?n,SEM=?s] -> IV[NUM=?n,SEM=?s]
VP[NUM=?n,SEM=[CORE=<?v(?obj)>, STORE=(?b1+?b2)]] -> TV[NUM=?n,SEM=[CORE=?v, STORE=?b1]] NP[SEM=[CORE=?obj, STORE=?b2]]
VP[NUM=?n,SEM=[CORE=<?v(?pp)(?obj)>, STORE=(?b1+?b2+?b3)]] -> DTV[NUM=?n,SEM=[CORE=?v, STORE=?b1]] NP[SEM=[CORE=?obj, STORE=?b2]] PP[+TO,SEM=[CORE=?pp, STORE=?b3]]

PP[+TO, SEM=[CORE=?np, STORE=?b1]] -> P[+TO] NP[SEM=[CORE=?np, STORE=?b1]]
"""

fcfg_storage_lexicon = r"""
PropN[NUM=sg,SEM=[CORE=<\P.P(angus)>, STORE=(/)]] -> 'Angus'
PropN[NUM=sg,SEM=[CORE=<\P.P(cyril)>, STORE=(/)]] -> 'Cyril'
PropN[NUM=sg,SEM=[CORE=<\P.P(irene)>, STORE=(/)]] -> 'Irene'

Det[NUM=sg,SEM=[CORE=<\P Q.all x.(P(x) -> Q(x))>, STORE=(/)]] -> 'every'
Det[NUM=sg,SEM=[CORE=<\P Q.exists x.(P(x) & Q(x))>, STORE=(/)]] -> 'a'

N[NUM=sg,SEM=[CORE=<\x.library(x)>, STORE=(/)]] -> 'library'
N[NUM=sg,SEM=[CORE=<\x.girl(x)>, STORE=(/)]] -> 'girl'
N[NUM=sg,SEM=[CORE=<\x.boy(x)>, STORE=(/)]] -> 'boy'
N[NUM=sg,SEM=[CORE=<\x.book(x)>, STORE=(/)]] -> 'book'

IV[NUM=sg,SEM=[CORE=<\x.smile(x)>, STORE=(/)],TNS=pres] -> 'smiles' 

TV[NUM=sg,SEM=[CORE=<\X x.X(\y.read(x,y))>, STORE=(/)],TNS=pres] -> 'reads'

DTV[NUM=sg,SEM=[CORE=<\Y X x.X(\z.Y(\y.give(x,y,z)))>, STORE=(/)],TNS=pres] -> 'gives'

P[+to] -> 'to'
""" 

fcfg_storage_np = r"""
NP[NUM=?n,SEM=[CORE=?np, STORE=?b1]] -> PropN[NUM=?n,SEM=[CORE=?np, STORE=?b1]]
NP[NUM=?n,SEM=[CORE=<?det(?nom)>, STORE=(?b1+?b2)]] -> Det[NUM=?n,SEM=[CORE=?det, STORE=?b1]] Nom[NUM=?n,SEM=[CORE=?nom, STORE=?b2]]
"""

sentences = [
    'Angus reads a book',
    'every girl reads a book',
    'every library gives a book to a girl',
]

fcfg_storage = fcfg_storage_base + fcfg_storage_np + fcfg_storage_lexicon
cs_syntax = FeatureGrammar.fromstring(fcfg_storage)
sents_reps = sem_parser(sentences, cs_syntax, verbose=False, is_cs=True)

for i, (sent, semreps) in enumerate(sents_reps.items()):
    counter = 0
    print(f"{i+1}. {sent}")
    for semrep in semreps:
        counter += 1
        display_translation(counter, semrep)

With the following change to the `NP` rule we push representations to the `STORE` and replace them with a simple varible in the `CORE` (which is then type raised to <<e,t>,t> so that it it can combine with a transitive verb entry).

In [ ]:
fcfg_storage_np = r"""
NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?np, @x)>+?b1)]] -> PropN[NUM=?n,SEM=[CORE=?np, STORE=?b1]]
NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?det(?nom), @x)>+?b1+?b2)]] -> Det[NUM=?n,SEM=[CORE=?det, STORE=?b1]] Nom[NUM=?n,SEM=[CORE=?nom, STORE=?b2]]
"""

sentences = [
    'Angus reads a book',
    'every girl reads a book',
    'every library gives a book to a girl',
]

fcfg_storage = fcfg_storage_base + fcfg_storage_np + fcfg_storage_lexicon
cs_syntax = FeatureGrammar.fromstring(fcfg_storage)
sents_reps = sem_parser(sentences, cs_syntax, verbose=False, is_cs=True)

for i, (sent, semreps) in enumerate(sents_reps.items()):
    counter = 0
    print(f"{i+1}. {sent}")
    for semrep in semreps:
        counter += 1
        display_translation(counter, semrep)

#### 2.2. Questions

**2a.** There are two identical readings of the first sentence. Why is this so? Suggest a change that would ensure you do not get doublicate readings for the first sentence but you get  alternative readings for the other sentence. **[2 marks]**

In [ ]:
sentences = [
    'Angus reads a book',
    'every girl reads a book'
]

fcfg_storage_np = r"""
## Change the rule below:
NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?np, @x)>+?b1)]] -> PropN[NUM=?n,SEM=[CORE=?np, STORE=?b1]]
NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?det(?nom), @x)>+?b1+?b2)]] -> Det[NUM=?n,SEM=[CORE=?det, STORE=?b1]] Nom[NUM=?n,SEM=[CORE=?nom, STORE=?b2]]
"""


fcfg_storage = fcfg_storage_base + fcfg_storage_np + fcfg_storage_lexicon 
cs_syntax = FeatureGrammar.fromstring(fcfg_storage)
sents_reps = sem_parser(sentences, cs_syntax, verbose=False, is_cs=True)

for i, (sent, semreps) in enumerate(sents_reps.items()):
    counter = 0
    print(f"{i+1}. {sent}")
    for semrep in semreps:
        counter += 1
        display_translation(counter, semrep)

**2b.** Extend the grammar below to cover the following sentences: **[6 marks]**

In [ ]:
sentences = [
    'every library gives a book to every girl and every boy',
    'no library gives every book to a boy',
    'a boy and a girl read all books',
    'Angus and Irene read all books',
]

# your answers
fcfg_storage_answers_1 = r"""
### Replace X with their proper representations
X -> 'no'
X -> 'all'
X -> 'books'
X -> 'read'
"""

fcfg_storage_answers_2 = r"""
### Correct the conjunction rule here. Replace xxx and ??? with the correct answer:
NP[NUM=xxx, SEM=[CORE=<???>, STORE=(?b1+?b2)]] -> NP[NUM=?num1, SEM=[CORE=?n1, STORE=?b1]] CONJ NP[NUM=?num2, SEM=[CORE=?n2, STORE=?b2]]
CONJ -> 'and'
"""


fcfg_storage = fcfg_storage_base + fcfg_storage_np + fcfg_storage_lexicon + fcfg_storage_answers_1 + fcfg_storage_answers_2
cs_syntax = FeatureGrammar.fromstring(fcfg_storage)
sents_reps = sem_parser(sentences, cs_syntax, verbose=False, is_cs=True)

for i, (sent, semreps) in enumerate(sents_reps.items()):
    counter = 0
    print(f"{i+1}. {sent}")
    for semrep in semreps:
        counter += 1
        display_translation(counter, semrep)

**2c.** Add the quantified expressions to the `STORE` only in the conjunction rule. Compare the number of readings you get now for the sentence below. Are there any invalid readings? If so, why? **[3 marks]**

In [ ]:
sentences = [
    'every library gives a book to every girl and every boy',
]

fcfg_storage_answers_2 = r"""
### Correct the conjunction rule here. Replace xxx and ??? with the correct answer:
NP[NUM=xxx, SEM=[CORE=<\P.P(@x)>, STORE=((<bo(???, @x)>)+?b1+?b2)]] -> NP[NUM=?num1, SEM=[CORE=?n1, STORE=?b1]] CONJ NP[NUM=?num2, SEM=[CORE=?n2, STORE=?b2]]
CONJ -> 'and'
"""


fcfg_storage = fcfg_storage_base + fcfg_storage_np + fcfg_storage_lexicon + fcfg_storage_answers_1 + fcfg_storage_answers_2
cs_syntax = FeatureGrammar.fromstring(fcfg_storage)
sents_reps = sem_parser(sentences, cs_syntax, verbose=False, is_cs=True)

for i, (sent, semreps) in enumerate(sents_reps.items()):
    counter = 0
    print(f"{i+1}. {sent}")
    for semrep in semreps:
        counter += 1
        display_translation(counter, semrep)

## Marks

This part of the assignment has a total of 14 marks.